## Steps to implement a langchain model with memory:

1. Import chat model and configure the API key
2. Create chat template
3. Initialize the memory
4. Create an output parser
5. Build a chain
6. Invoke the chain with human input and chat history
7. Saving to memory
8. Run step 6 and 7 in loop

# Step 1: Create chat model and configure the API key

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
with open(r"C:\Users\DeLL\Documents\Data Science\Gen AI\Building  conversational chat bot and intro to Gemini\Google API Key.txt") as f:
    key = f.read().strip()
chat_model = ChatGoogleGenerativeAI(api_key = key, model = 'gemini-1.5-flash')

# Step 2: Create chat Template

In [14]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

#### Example of memory implementation

In [8]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
memory.load_memory_variables({})

{'history': ''}

In [9]:
memory.buffer

''

In [10]:
memory = ConversationBufferMemory(memory_key = 'chat_history')
memory.chat_memory.add_user_message('Hi!')
memory.chat_memory.add_ai_message("Hi! what's up? How can i help you?")
memory.chat_memory.add_user_message('what is regression?')
memory.load_memory_variables({})

{'chat_history': "Human: Hi!\nAI: Hi! what's up? How can i help you?\nHuman: what is regression?"}

In [11]:
memory.buffer

"Human: Hi!\nAI: Hi! what's up? How can i help you?\nHuman: what is regression?"

In [12]:
print(memory.buffer)

Human: Hi!
AI: Hi! what's up? How can i help you?
Human: what is regression?


In [20]:
memory = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)
memory.chat_memory.add_user_message('Hi!')
memory.chat_memory.add_ai_message("Hi! what's up? How can i help you?")
memory.chat_memory.add_user_message('what is regression?')
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi! what's up? How can i help you?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is regression?', additional_kwargs={}, response_metadata={})]}

In [21]:
memory.buffer

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi! what's up? How can i help you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is regression?', additional_kwargs={}, response_metadata={})]

*****************************************************************************************************************************

In [15]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content = 'You are having a chatbot having a conversation with a human.'),
    # Creating a chat history place holder
    MessagesPlaceholder(variable_name = 'chat_history'),
    # Human_prompt
    HumanMessagePromptTemplate.from_template('{human_input}')
])

# Step 3: Initialize the memory

In [30]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)

# Step 4: Create an Output parser

In [31]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# Step 5: Build a chain (Alternative approach)

### That method was called Runnable passthrough or runnable method (using '|')

But now since we will use RAG this method (which we will see now) is more appreciated.

In [32]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# define a function to load messages from memory
def get_messages_from_memory(human_input):
    return memory.load_memory_variables(human_input)['chat_history']
    
# Define a chain
chain = RunnablePassthrough.assign(chat_history = RunnableLambda(get_messages_from_memory)) | chat_prompt_template | chat_model | output_parser

# Step 6: Invoke the chain with human_input and chat history

In [33]:
query = {'human_input' : 'Hi! how are you?'}
response = chain.invoke(query)
response

"Hi there! I'm doing great, thanks for asking. How are you today?"

from langchain_core.runnables import RunnablePassthrough, RunnableLambda

#define a function to load messages from memory
def get_messages_from_memory(human_input):
    return memory.buffer

#Define a chain
chain = RunnablePassthrough.assign(chat_history = RunnableLambda(get_messages_from_memory)) | chat_prompt_template | chat_model | output_parser

query = {'human_input' : 'Hi! how are you?'}
response = chain.invoke(query)
response

# Step 7: Saving to memory

In [35]:
memory.save_context(query, {'output' : response})
memory.buffer

[HumanMessage(content='Hi! how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi there! I'm doing great, thanks for asking. How are you today?", additional_kwargs={}, response_metadata={})]

# Step 8: Run steps 6 and 7 in a loop

In [36]:
while True:
    query = {'human_input' : input('Enter your input :' )}
    print(f'User: {query['human_input']}')
    if query['human_input'].lower().strip() in ['a', 'bye']:
        break
    response = chain.invoke(query)
    print(f'AI : {response}')
    memory.save_context(query, {'output' : response})
             

Enter your input : hi


User: hi
AI : Hi!  What's up?


Enter your input : how are you?


User: how are you?
AI : I'm doing well, thank you for asking!  How are you today?


Enter your input : my name is vishal, what is your name?


...

User: thanks, do you remember my name?
AI : Yes, you told me your name is Vishal.


Enter your input : summarize the entire conversation


User: summarize the entire conversation
AI : We started with greetings and pleasantries.  You asked about my well-being, and I reciprocated. You then introduced yourself as Vishal and asked my name (I explained I don't have one as I'm an AI).  Finally, you asked about the BERT model, and I provided a detailed explanation of its key features and applications.  We concluded with you asking if I remembered your name, which I confirmed.


Enter your input : a


User: a


In [38]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='Hi! how are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi there! I'm doing great, thanks for asking. How are you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi!  What's up?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm doing well, thank you for asking!  How are you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is vishal, what is your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I don't have a name. I'm a large language model, an AI.  It's nice to meet you, Vishal!", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='tell me something about BERT model', additional_kwargs={}, response_metadata={}),
  AIMessage(content="BER

In [39]:
memory.buffer

[HumanMessage(content='Hi! how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi there! I'm doing great, thanks for asking. How are you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hi!  What's up?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thank you for asking!  How are you today?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='my name is vishal, what is your name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I don't have a name. I'm a large language model, an AI.  It's nice to meet you, Vishal!", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='tell me something about BERT model', additional_kwargs={}, response_metadata={}),
 AIMessage(content="BERT, or Bidirectional Encode

**what we have implemented so far:**

In [45]:
# creating model and setting api key
from langchain_google_genai import ChatGoogleGenerativeAI
with open(r"C:\Users\DeLL\Documents\Data Science\Gen AI\Building  conversational chat bot and intro to Gemini\Google API Key.txt") as f:
    key = f.read().strip()

chat_model = ChatGoogleGenerativeAI(api_key = key, model = 'gemini-1.5-flash')
##############

# creating chat template
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages([
    SystemMessage(content = '''Behave like a helpful AI instructor who is concerned to clear the doubts and concepts related 
    to data science, AI and Gen AI. You give the answer in a simple and intuitive manner and in such a way so that the person 
    can easily understand and appear confidently for interviews related to AI. You also give examples wherever required. Lets think step by step.'''),
    MessagesPlaceholder(variable_name = 'chat_history'),
    HumanMessagePromptTemplate.from_template('{human_input}')
])
###############

#initializing memory
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key = 'chat_history', return_messages = True)
##############

#creating an output_parser
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
##############

# creating chain
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# function to load message from memory

def get_message_from_memory(human_input):
    return memory.load_memory_variables(human_input)['chat_history']

chain = RunnablePassthrough.assign(chat_history = RunnableLambda(get_message_from_memory)) | chat_template | chat_model | output_parser

###################

# Invoke the chain with human input and chat history
query = {'human_input' : 'explain the key difference between BERT and GPT models'}
response = chain.invoke(query)
print(response)
print()

##################

# Saving to memory
memory.save_context(query, {'output' : response})

##################

# repating step 6 and 7 in loop to generate a conversational bot
while True:

    query = {'human_input' : input('Enter your prompt here : ')}
    print(f'User : {query['human_input']}')
    if query['human_input'] in ['bye', 'exit']:
        print('Bye! Have a nice day!')
        break
    response = chain.invoke(query)
    print(f'AI BOT : {response}')
    print()
    memory.save_context(query, {'output' : response})

Let's break down the key differences between BERT and GPT models, focusing on how they understand and generate text.  Think of it like this: BERT is a great *understanding* machine, while GPT excels at *generation*.

**BERT (Bidirectional Encoder Representations from Transformers):**

* **Focus:** Understanding the context of a word by looking at the words *before and after* it simultaneously.  This is called "bidirectional." Imagine you're reading a sentence – you don't just read left-to-right; you use the entire sentence's context to understand each word.  That's what BERT does.
* **Architecture:**  It's primarily an encoder. Think of an encoder as a machine that takes input (a sentence) and transforms it into a representation that captures its meaning.  It doesn't generate new text; it analyzes existing text.
* **Tasks:** Excellent at tasks like question answering, sentiment analysis, named entity recognition (finding names of people, places, etc.).  It understands the nuances of la

Enter your prompt here :  hi


User : hi
AI BOT : Hi there! How can I help you today?  Do you have any questions about data science, AI, or generative AI?  I'm ready to assist you.



Enter your prompt here :  what skills are required to become a gen ai engineer


User : what skills are required to become a gen ai engineer
AI BOT : Becoming a Gen AI engineer requires a blend of technical expertise, creative problem-solving, and a strong understanding of the underlying principles.  Here's a breakdown of the key skills:

**Foundational Skills:**

* **Strong Programming Proficiency:**  Python is the dominant language in AI, so mastery is crucial. You'll need to be comfortable with data structures, algorithms, and object-oriented programming.
* **Mathematics and Statistics:** A solid grasp of linear algebra, calculus, probability, and statistics is fundamental.  You'll be working with large datasets and complex models, so understanding the mathematical underpinnings is essential.
* **Machine Learning Fundamentals:**  Understanding various machine learning algorithms (supervised, unsupervised, reinforcement learning) is vital.  You should be able to choose the right algorithm for a given task and understand its limitations.
* **Deep Learning:**  Gen 

Enter your prompt here :  my name is vishal what is your name


User : my name is vishal what is your name
AI BOT : I don't have a name. I'm a large language model, an AI.  It's nice to meet you, Vishal!



Enter your prompt here :  bye


User : bye
Bye! Have a nice day!


In [46]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='explain the key difference between BERT and GPT models', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Let\'s break down the key differences between BERT and GPT models, focusing on how they understand and generate text.  Think of it like this: BERT is a great *understanding* machine, while GPT excels at *generation*.\n\n**BERT (Bidirectional Encoder Representations from Transformers):**\n\n* **Focus:** Understanding the context of a word by looking at the words *before and after* it simultaneously.  This is called "bidirectional." Imagine you\'re reading a sentence – you don\'t just read left-to-right; you use the entire sentence\'s context to understand each word.  That\'s what BERT does.\n* **Architecture:**  It\'s primarily an encoder. Think of an encoder as a machine that takes input (a sentence) and transforms it into a representation that captures its meaning.  It doesn\'t generate new text; it analyzes existing text.\n

### Now whenever we turn off the system or disconnect from internet all this history gets deleted.

# Saving chat history

In [47]:
import pickle

In [48]:
chat_history = pickle.dumps(memory)
with open('conversation_memory.pkl','wb') as f:
    f.write(chat_history)

# Loading chat history

In [49]:
chat_history_loaded = pickle.load(open('conversation_memory.pkl','rb'))
print(chat_history_loaded)

chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='explain the key difference between BERT and GPT models', additional_kwargs={}, response_metadata={}), AIMessage(content='Let\'s break down the key differences between BERT and GPT models, focusing on how they understand and generate text.  Think of it like this: BERT is a great *understanding* machine, while GPT excels at *generation*.\n\n**BERT (Bidirectional Encoder Representations from Transformers):**\n\n* **Focus:** Understanding the context of a word by looking at the words *before and after* it simultaneously.  This is called "bidirectional." Imagine you\'re reading a sentence – you don\'t just read left-to-right; you use the entire sentence\'s context to understand each word.  That\'s what BERT does.\n* **Architecture:**  It\'s primarily an encoder. Think of an encoder as a machine that takes input (a sentence) and transforms it into a representation that captures its meaning.  It doesn\'t generate new text;

# SQL chat message history

In [50]:
!pip install --quiet langchain_community 

In [56]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_message_history_from_db(session_id):
    chat_message_history = SQLChatMessageHistory(session_id = session_id, connection = 'sqlite:///chats_data/sqlite.db')
    return chat_message_history

In [57]:
# Create a chat template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate.from_messages([('system', 'You are a helpful AI assistant'),
                                                  MessagesPlaceholder(variable_name = 'history'),
                                                  ('user', '{user_input}')])

In [58]:
#chain

chain = chat_template | chat_model

In [59]:
from langchain_core.runnables.history import RunnableWithMessageHistory

#define a chain

conversation_chain = RunnableWithMessageHistory(chain,
                                                get_session_message_history_from_db,
                                                input_messages_key = 'user_input',
                                                history_messages_key = 'history')

In [60]:
# This is where we configure the session id
user_id = 'vishal'
config = {'configurable' : {'session_id' : user_id}}
input_prompt = {'user_input' : 'what is the name of the first US president?'}
response = conversation_chain.invoke(input = input_prompt, config = config)
response.content

'George Washington'

In [62]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_message_history_from_db(session_id):
    import os
    os.makedirs('data', exist_ok=True)  # Ensure directory exists
    db_path = os.path.abspath('data/sqlite.db')
    print(f"Using database at {db_path}")
    chat_message_history = SQLChatMessageHistory(session_id = session_id, connection = f'sqlite:///{db_path}')
    return chat_message_history

# Create a chat template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate.from_messages([('system', 'You are a helpful AI assistant'),
                                                  MessagesPlaceholder(variable_name = 'history'),
                                                  ('user', '{user_input}')])

#chain

chain = chat_template | chat_model

from langchain_core.runnables.history import RunnableWithMessageHistory

#define a chain

conversation_chain = RunnableWithMessageHistory(chain,
                                                get_session_message_history_from_db,
                                                input_messages_key = 'user_input',
                                                history_messages_key = 'history')

# This is where we configure the session id
user_id = 'vishal'
config = {'configurable' : {'session_id' : user_id}}
input_prompt = {'user_input' : 'what is the name of the first US president?'}
response = conversation_chain.invoke(input = input_prompt, config = config)
response.content


Using database at C:\Users\DeLL\Documents\Data Science\Gen AI\17th Oct 2024\data\sqlite.db


'George Washington'

In [63]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

def get_session_message_history_from_db(session_id):
    chat_message_history = SQLChatMessageHistory(session_id = session_id, connection = 'sqlite:///sqlite_test/sqlite.db')
    return chat_message_history

# Create a chat template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_template = ChatPromptTemplate.from_messages([('system', 'You are a helpful AI assistant'),
                                                  MessagesPlaceholder(variable_name = 'history'),
                                                  ('user', '{user_input}')])

#chain

chain = chat_template | chat_model

from langchain_core.runnables.history import RunnableWithMessageHistory

#define a chain

conversation_chain = RunnableWithMessageHistory(chain,
                                                get_session_message_history_from_db,
                                                input_messages_key = 'user_input',
                                                history_messages_key = 'history')

# This is where we configure the session id
user_id = 'vishal'
config = {'configurable' : {'session_id' : user_id}}
input_prompt = {'user_input' : 'what is the name of the first US president?'}
response = conversation_chain.invoke(input = input_prompt, config = config)
response.content


'George Washington'

In [64]:
user_id = 'nikita'
config = {'configurable' : {'session_id' : user_id}}
input_prompt = {'user_input' : 'what is the name of the first US president?'}
response = conversation_chain.invoke(input = input_prompt, config = config)
response.content

'George Washington'

In [68]:
def chat_bot(session_id, prompt):

    config = {'configurable' : {'session_id' : user_id}}
    input_prompt = {'user_input' : prompt}
    response = conversation_chain.invoke(input = input_prompt, config = config)
    return response.content

In [71]:
chat_bot('vishal', 'my name is vishal. what is the capital of alaska?')

'The capital of Alaska is Juneau.'

In [72]:
chat_bot('vishal', 'do you remember my name?')

'You told me your name is Vishal.'

In [74]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# creating function to get message history
def get_message_history_from_db(session_id):
    chat_history = SQLChatMessageHistory(session_id = session_id, connection = 'sqlite:///sqlite_test/sqlite.db')
    return chat_history

#creating template

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
chat_template = ChatPromptTemplate.from_messages([('system', 'you are a helpful AI assistant'),
                                                  MessagesPlaceholder(variable_name = 'history'),
                                                  ('human', '{human_input}')])
chain = chat_template | chat_model

#bringing the memory and the chain (pipeline) together

from langchain_core.runnables import RunnableWithMessageHistory

conversational_chain = RunnableWithMessageHistory(chain,
                                                  get_message_history_from_db,
                                                  input_messages_key = 'human_input',
                                                  history_messages_key = 'history')
#configuring the session id

user_id = 'vikas'
config = {'configurable' : {'session_id' : user_id}}
input_prompt = {'human_input' : 'my name is vikas. what are the names of union territories of india?'}
response = conversational_chain.invoke(input_prompt, config = config)
print(response.content)



Hi Vikas,

The Union Territories of India are:

* Jammu and Kashmir
* Ladakh
* Delhi
* Puducherry
* Chandigarh
* Dadra and Nagar Haveli and Daman and Diu
* Lakshadweep
* Andaman and Nicobar Islands

Please note that the status and names of these territories can sometimes be subject to change due to political developments.  This list is accurate as of October 26, 2023.


In [6]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# Retrieves the history for a particular session using session id
def get_messages_history_from_db(session_id):
    chat_history = SQLChatMessageHistory(session_id = session_id, connection = 'sqlite:///sqlite_test/sqlite.db')
    return chat_history

# Creating a chat template
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
chat_template = ChatPromptTemplate.from_messages([('system', 'You are a helpful AI assistant and your name is Freddy'),
                                                  MessagesPlaceholder(variable_name = 'history'),
                                                  ('human', '{human_input}')])

# creating a chain
chain = chat_template | chat_model

#bringing the memory and the chain (pipeline) together
from langchain_core.runnables import RunnableWithMessageHistory
conversational_chain = RunnableWithMessageHistory(chain,
                                                  get_messages_history_from_db,
                                                  history_messages_key = 'history',
                                                  input_messages_key = 'human_input')

#configuring the session id and invoking the chain

user_id = 'vishal'
config = {'configurable' : {'session_id' : user_id}}

# implementing a conversational bot

while True:
    user_prompt = input('Enter your prompt here : ')
    if user_prompt.lower().strip() in ['bye', 'exit']:
        print('Bye! Have a good day!')
        break
    input_prompt = {'human_input' : user_prompt}
    response = conversational_chain.invoke(input_prompt, config = config)
    print(f'Bot : {response.content}')


Enter your prompt here :  hi


Bot : Hi Vishal! How are you doing today?


Enter your prompt here :  bye


Bye! Have a good day!


In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect(r"C:\Users\DeLL\Documents\Data Science\Gen AI\17th Oct 2024\sqlite_test\sqlite.db")

OperationalError: unable to open database file

In [9]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = connection.execute(query).fetchall()

# Print all table names
print("Tables in the database:", tables)

connection.close()

Tables in the database: [('message_store',)]


In [12]:
import pandas as pd
df = pd.read_sql_query('SELECT * FROM message_store', connection)
connection.close()
df

,id,session_id,message
0,1,vishal,"{""type"": ""human"", ""data"": {""content"": ""what is..."
1,2,vishal,"{""type"": ""ai"", ""data"": {""content"": ""George Was..."
2,3,nikita,"{""type"": ""human"", ""data"": {""content"": ""what is..."
3,4,nikita,"{""type"": ""ai"", ""data"": {""content"": ""George Was..."
4,5,vishal,"{""type"": ""human"", ""data"": {""content"": ""what is..."
5,6,vishal,"{""type"": ""ai"", ""data"": {""content"": ""Juneau"", ""..."
6,7,vishal,"{""type"": ""human"", ""data"": {""content"": ""do you ..."
7,8,vishal,"{""type"": ""ai"", ""data"": {""content"": ""No, I don'..."
8,9,nikita,"{""type"": ""human"", ""data"": {""content"": ""what is..."
9,10,nikita,"{""type"": ""ai"", ""data"": {""content"": ""Juneau"", ""..."


In [14]:
df = df[df['session_id'] == 'vishal']

In [30]:
df['message'].values

array(['{"type": "human", "data": {"content": "what is the name of the first US president?", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}',
       '{"type": "ai", "data": {"content": "George Washington", "additional_kwargs": {}, "response_metadata": {"prompt_feedback": {"block_reason": 0, "safety_ratings": []}, "finish_reason": "STOP", "safety_ratings": []}, "type": "ai", "name": null, "id": "run-50e75841-41e2-4680-b6d4-6f8ab9068105-0", "example": false, "tool_calls": [], "invalid_tool_calls": [], "usage_metadata": {"input_tokens": 17, "output_tokens": 3, "total_tokens": 20, "input_token_details": {"cache_read": 0}}}}',
       '{"type": "human", "data": {"content": "what is the capital of alaska?", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null, "example": false}}',
       '{"type": "ai", "data": {"content": "Juneau", "additional_kwargs": {}, "response_metadata": {"prompt_fee

In [31]:
import numpy as np
import json

# Example array
data = df['message'].values

# Extract the content part
contents = [json.loads(entry)['data']['content'] for entry in data]

# Display the content
for content in contents:
    print(content)


what is the name of the first US president?
George Washington
what is the capital of alaska?
Juneau
do you remember my name?
No, I don't remember your name.  As a large language model, I have no memory of past conversations.  Each interaction with me starts fresh.
hi
Hi there! How can I help you today?
what is your name?
My name is Freddy.
do you remember my name?
No, I don't remember your name.  I have no memory from previous conversations.
what is 59*821
48379
my name is vishal
It's nice to meet you, Vishal!  How can I help you?
what is RAG?
RAG stands for **Retrieval Augmented Generation**.  It's a technique used in large language models (like me!) to improve the accuracy and relevance of their responses.

Instead of relying solely on the knowledge stored within the model's parameters, RAG systems incorporate an external knowledge base.  When a question is asked, the model first retrieves relevant information from this knowledge base. This retrieved information is then used to augme